# DeepDream
## Inception V3 


In [1]:
import keras

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
keras.__version__

'2.2.2'

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15043440082116597145
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15589952717
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1723042628417857993
physical_device_desc: "device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:00:1e.0, compute capability: 7.0"
]


### Load pretrained Inception V3 model from keras

In [4]:
from keras.applications import inception_v3
from keras import backend as K
# disable model training
K.set_learning_phase(0)

In [5]:
# build Inception V3 network without its convolutional base. 
# Load model with pretrained ImageNet weights
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

### Set up the DeepDream configuration

Compute the loss to maximize during the *gradient-ascent* process

in parallel, maximize the activation of all filters in a number of layers and the weighted sum of L2 norm of activations of a set of high-level layers

Lower levels = Geometric patterns
Higher levels = visuals containing class features from ImageNet (dog, cat etc)

#### Initial config: 4 Layers
Create a dictionary mapping of layer names and a coefficient quantifying the degree of the layer's activation contribution to the loss we seek to minimize. *Layer names are hard-coded in Inception, use model.summary() for all layer names to create varying configs.

In [6]:
layer_contributions = {
    'mixed1': 3.0, 'mixed2': 3., 'mixed3': 2.0, 'mixed4': 3., 'mixed5': 1.5, 'mixed6': 3., 'mixed7': 2.0, 
    'mixed8': 2.2, 'mixed9': 2.0
}

#### Define a tensor containing the loss: the weighted sum of the L2 norm of the activations of the layers

##### Define the loss to be maximized

Create a dictionary mapping of layer names to layer instances

Define loss by adding layer contributions to the scalar variable

Retrieve layer's output to activation variable

Add the L2 norm of the features of a layer to the loss. Border artifacts are avoided by only involving nonporder pixels in the loss

In [7]:
layer_dict = dict(
    [layer.name, layer] for layer in model.layers
)

In [8]:
loss = K.variable(0.)

In [9]:
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss += coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling
    

### Define Gradient-ascent process

In [10]:
# tensor to hold the generated image
dream = model.input

# compute gradients of the dream with regard to the loss
grads = K.gradients(loss, dream)[0]

# normalise gradients
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

# setup a keras function to retrieve the value of the loss and gradients, given an input image
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grads_value = outs[1]
    return loss_value, grads_value

# gradient ascent function to run for a number of iterations
def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_value = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('... Loss value at {}, step:{}'.format(loss_value, i))
        x += step * grad_value
    return x


## DeepDream Algorithm

1 - Define a list of *scales* (octaves) to process the images

2 - Each successive scale is larger than the previous one by a factor of 1.4 (40%),
    Start by processing a small image and increasingly scale up
    
3 - For each successive scale, from smallest to largets, run a gradient descent to maximize the loss previously defined     at that scale.

4 - After each gradient ascent run, upscale the resulting image by 40%

### Detail injection

To avoid losing detail at each scale up, at each scale up, reinject the loss details back into the image using the original

### Run gradient ascent over different successive scales

In [11]:
import numpy as np

# hyperparameters for varying effects
step = 0.01 # gradient ascent step
num_octave = 3  # number of scales to run gradient ascent
octave_scale = 1.4  # size ratio between scales
iterations = 100  # number of iteration steps to run at each scale

# max loss to interrup the gradient ascent process to avoid image artifacts
max_loss = 10.0

In [12]:
import scipy
from keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)


def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    scipy.misc.imsave(fname, pil_img)


def preprocess_image(image_path):
    # Util function to open, resize and format pictures
    # into appropriate tensors.
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img


def deprocess_image(x):
    # Util function to convert a tensor into a valid image.
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

### Preprocess image

Loads base image into a numpy array

Prepares a list of shape tuples defining the different scale to run gradient ascent

Reverse the list of shapes so theyre in increasing order

Resize numpy array of the image to the smallest scale

Scale up smaller version of the orinigal image

Compute high quality version of original image

Compute difference in detail lost between images when scaling up

Reinject lost detail into the dream

In [13]:
# base image path for sample input
base_image_path = 'images/florence_1.jpg'

In [14]:
str(base_image_path[:-4])

'images/florence_1'

In [15]:
img = preprocess_image(base_image_path)

In [16]:
img

array([[[[ 0.11372554,  0.24705887,  0.5764706 ],
         [ 0.10588241,  0.23921573,  0.5686275 ],
         [ 0.09019613,  0.22352946,  0.5529412 ],
         ...,
         [ 0.20000005,  0.05882359, -0.06666666],
         [ 0.14509809,  0.00392163, -0.10588235],
         [ 0.16078436,  0.03529418, -0.06666666]],

        [[ 0.12156868,  0.254902  ,  0.58431375],
         [ 0.13725495,  0.27058828,  0.6       ],
         [ 0.11372554,  0.24705887,  0.5764706 ],
         ...,
         [ 0.18431377,  0.04313731, -0.08235294],
         [ 0.19215691,  0.05882359, -0.05098039],
         [ 0.14509809,  0.0196079 , -0.08235294]],

        [[ 0.12156868,  0.254902  ,  0.58431375],
         [ 0.11372554,  0.24705887,  0.5764706 ],
         [ 0.11372554,  0.24705887,  0.5764706 ],
         ...,
         [ 0.20784318,  0.06666672, -0.05098039],
         [ 0.20000005,  0.05882359, -0.04313725],
         [ 0.1686275 ,  0.04313731, -0.06666666]],

        ...,

        [[-0.8509804 , -0.8352941 , -0

In [17]:
original_shape = img.shape[1:3]

In [18]:
# prepare list of shape tuples defining the different scales to run gradient ascent
successive_shapes = [original_shape]

In [19]:
for i in range(1, num_octave):
    shape = tuple(
        [int(dim / (octave_scale ** i)) for dim in original_shape]
    )
    successive_shapes.append(shape)

In [20]:
successive_shapes

[(1672, 2508), (1194, 1791), (853, 1279)]

In [21]:
# reverse the list of shapes to increasing order
successive_shapes = successive_shapes[::-1]

In [22]:
successive_shapes

[(853, 1279), (1194, 1791), (1672, 2508)]

In [23]:
original_img = np.copy(img)

In [24]:
shrunk_original_img = resize_img(img, successive_shapes[0])

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:583: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


In [25]:
# scale up dream image
for shape in successive_shapes:
    print('Processing image shape ', shape)
    # resize image and run gradient ascent
    img = resize_img(img, shape)
    img = gradient_ascent(img, iterations=iterations, step=step, max_loss=max_loss)
    # scale up smaller version of the original image
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    # compute high quality version of original version of original size
    same_size_original = resize_img(original_img, shape)
    # compute lost detail
    lost_detail = same_size_original - upscaled_shrunk_original_img
    
    # reinject lost detail into dream
    img += lost_detail
    
    # resize original
    shrunk_original_img = resize_img(original_img, shape)
    # save each scale to file
    save_img(img, fname=str(base_image_path[:-4]) + 'dream_at_scale_' + str(shape) + '.png')


# save final image to file
save_img(img, fname=str(base_image_path[:-4]) + 'final_dream.png')
print('---- Dreams Complete')

Processing image shape  (853, 1279)


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:583: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


... Loss value at 6.50858211517334, step:0
... Loss value at 7.582132339477539, step:1
... Loss value at 8.713152885437012, step:2
... Loss value at 9.84902572631836, step:3


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:15: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Processing image shape  (1194, 1791)
... Loss value at 7.045809268951416, step:0
... Loss value at 8.602713584899902, step:1
... Loss value at 9.95887565612793, step:2
Processing image shape  (1672, 2508)
... Loss value at 7.142994403839111, step:0
... Loss value at 8.683769226074219, step:1
---- Dreams Complete
